In [1]:
DEFAULT_BASE_URL = "http://pipeline:8080/%s"

In [2]:
import pandas as pd
import os.path
import requests
from urllib.parse import urlencode
import json
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import tensorflow.python.framework.dtypes
# things we need for Tensorflow
import numpy as np
import tensorflow as tf
import random

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin



In [4]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data.parquet", "rb" ) )
# data = pd.read_parquet(os.path.join("data", "training_data.parquet"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [5]:
epochs = 1000
batch_size = 8

# Build neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(len(train_x[0]),)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(len(train_y[0]), activation="softmax"),
])

In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("looking for help", words)
print (p)
print (classes)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [8]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
userid =123


def score_chat(text, url = None):
    d = np.array([bow(sentence, words)])
    payload = urlencode({"json_args" : json.dumps(d)})
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    response = requests.request("POST", url, data=payload, headers=headers)
    try:
        return json.loads(response.text)
    except BaseException as e:
        raise RuntimeError("Error: caught %r while processing %r (%r)" % (e, response, response.text))


# @app.route("/redsam/api/v1.0/classify", methods=['POST'])
def classify(sentence):
    # generate probabilities from the model
   
    results = score_chat(sentence)[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID=userid, show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        print(random.choice(i['responses']))
                    return

            results.pop(0)

In [10]:

score_chat("what help you offer?")

hello I don't know how to score the args (['what help you offer?']) you supplied


"I don't know how to score the args (['what help you offer?']) you supplied"